In [1]:
import os
# Skill issue de windows
if os.name == "nt":
    from pathlib import Path
    print(os.getcwd())
    script_dir = Path().resolve().parent
    os.chdir(script_dir)
    print(os.getcwd())


In [2]:
from TP2.pipline_comparar import comparar_modelos
from TP2 import reconstruccion_textocatedra
from TP2.models.simple_gru import SimpleGru
import torch
from TP2.pipline_comparar import comparar_modelos


/home/jorge/oldhome/Documents/UBA/Carrera/2C2025/AprendizajeAutomaticoArgentino/aprendizaje_automatico/TP2/pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\{'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
/home/jorge/oldhome/Documents/UBA/Carrera/2C2025/AprendizajeAutomaticoArgentino/aprendizaje_automatico/TP2/pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\m'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
/home/jorge/.conda/envs/aa_dc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
PATH = ""
PATH_DATASET_PARA_COMPARAR = ""
PATH_DATASET_TEXTO_CATEDRA = ""
PATH_DATASET_MATRIZ_DE_CONFUSION = PATH_DATASET_TEXTO_CATEDRA


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_441371/2200634214.py:2: SyntaxWarning: invalid escape sequence '\d'
  PATH_DATASET_PARA_COMPARAR = "TP2\dataset\guardar_datos"


Agarro mejor modelo

In [14]:
modelos = {
    #"RNNBidireccional": RNN_LSTM(embedding_dim=768, hidden_dim=256, num_layers=2, bidirectional=True),
    "SimpleGru" : SimpleGru(embedding_dim=768, hidden_dim=256, num_layers=3, bidirectional=True)
}
# hay que hacer el load de cada modelo acá!
print('cuda' if torch.cuda.is_available() else 'cpu')
resultados = comparar_modelos(PATH_DATASET_PARA_COMPARAR, modelos)

#me quedo con el mejor modelo

mejor_modelo_nombre = min(resultados, key=lambda k: resultados[k]['avg_loss']) #esto deberia hacerse ponderando
mejor_modelo = modelos[mejor_modelo_nombre]

mejor_modelo 


cpu


/home/jorge/oldhome/Documents/UBA/Carrera/2C2025/AprendizajeAutomaticoArgentino/aprendizaje_automatico/TP2/pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\{'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))
/home/jorge/oldhome/Documents/UBA/Carrera/2C2025/AprendizajeAutomaticoArgentino/aprendizaje_automatico/TP2/pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\m'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))


FileNotFoundError: [Errno 2] No such file or directory: 'TP2\\models\\DictMejores\\SimpleGru_best_model.pth'

Reconstruyo texto de catedra

In [4]:
mejor_modelo = SimpleGru(embedding_dim=768, hidden_dim=256, num_layers=3, bidirectional=True)
mejor_modelo.load_state_dict(torch.load(PATH))
mejor_modelo.eval()

SimpleGru(
  (lstm): GRU(768, 256, num_layers=3, batch_first=True, bidirectional=True)
  (punct_ini_fc): Linear(in_features=512, out_features=2, bias=True)
  (punct_fin_fc): Linear(in_features=512, out_features=4, bias=True)
  (cap_fc): Linear(in_features=512, out_features=4, bias=True)
)

In [5]:
from TP2.reconstruccion_textocatedra import recontruccion_texto_catedra
import pandas as pd
from TP2.tokenizer import PunctuationPredictor

df_catedra = pd.read_csv(PATH_DATASET_TEXTO_CATEDRA)
#instancianum, , token_id, token. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
texto_reconstruido = recontruccion_texto_catedra(df_catedra, mejor_modelo, device)
print(texto_reconstruido)
# instancianum, token_id, token, punt_inc  
#guardo el texto en un archivo csv
texto_reconstruido.to_csv("texto_reconstruido_catedra.csv", index=False)


#
##esto queda pendiente!!

       instancia_id  token_id      token punt_inicial punt_final  \
0                 0     10109         la                           
1                 0     67763   cuestión                           
2                 0     10196         es                           
3                 0     10121        que                           
4                 0     10109         la                           
...             ...       ...        ...          ...        ...   
90823          3964     21114      ##lam                           
90824          3964     10310       ##os                       .   
90825          3964       169          a                           
90826          3964     10109         la                           
90827          3964     43249  izquierda                       .   

       capitalización  
0                   1  
1                   0  
2                   0  
3                   0  
4                   0  
...               ...  
90823          

In [ ]:
import pandas as pd

from TP2.tokenizer import reconstruir_token_a_normalizada

recon = reconstruir_token_a_normalizada(texto_reconstruido)

PP = PunctuationPredictor(mejor_modelo)
for oracion in recon:
     print(f"Input: {oracion}")
     print(f"Output: {PP.predict(oracion)}")
     print()

In [7]:
import re
oracion = " cuándo vamos a mcdonald ' s ellos no vienen hoy dónde están ahora"
oracion = oracion[1:] 

print(oracion)
oracion_limpia = re.sub(r"\s' s", "'s", oracion)
print(oracion_limpia)

oracion2 = " él dijo que no ' ve a ella "
oracion2 = oracion2[1:]
print(oracion2)
oracion2_limpia = re.sub(r"\s' (\w+)", r"'\1", oracion2)
print(oracion2_limpia)


cuándo vamos a mcdonald ' s ellos no vienen hoy dónde están ahora
cuándo vamos a mcdonald's ellos no vienen hoy dónde están ahora
él dijo que no ' ve a ella 
él dijo que no've a ella 


## Matriz de confusion

In [8]:
def graficar_matrices_de_confusion(resultados):
    """
    Grafica las matrices de confusión para todos los atributos (puntuación inicial, 
    puntuación final y capitalización) de todos los modelos en los resultados.
    
    Args:
        resultados: dict con formato {'nombre_modelo': {'nombre_modelo': ..., 
                   'f1_punct_ini': ..., 'punct_ini_pred': [...], 
                   'punct_ini_ground_truth': [...], ...}, ...}
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix
    
    atributos = [
        ('punct_ini', 'Puntuación Inicial'),
        ('punct_fin', 'Puntuación Final'),
        ('cap', 'Capitalización')
    ]
    
    for nombre_modelo, metricas in resultados.items():
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        fig.suptitle(f'Matrices de Confusión - Modelo: {nombre_modelo}', fontsize=16, fontweight='bold')
        
        for idx, (atributo_key, atributo_label) in enumerate(atributos):
            # Obtener predicciones y valores reales
            y_true = metricas[f'{atributo_key}_ground_truth']
            y_pred = metricas[f'{atributo_key}_pred']
            
            # Calcular matriz de confusión
            cm = confusion_matrix(y_true, y_pred)
            
            # Graficar
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], 
                       cbar_kws={'label': 'Cantidad'})
            axes[idx].set_title(f'{atributo_label}', fontweight='bold')
            axes[idx].set_xlabel('Predicción')
            axes[idx].set_ylabel('Verdadero')
        
        plt.tight_layout()
        plt.show()


In [ ]:
from TP2.dataloader import TextDataset

dataloader_para_matriz = TextDataset(
    base_path=PATH_DATASET_MATRIZ_DE_CONFUSION,  parquet_file_subpath="tokens_batch_", batch_size=8192, instance_count=10000
    ) 

matriz_de_confusion_resultados = resultados

# Graficar todas las matrices de confusión
graficar_matrices_de_confusion(matriz_de_confusion_resultados)

D:\Facultad\AA\aprendizaje_automatico\TP2\pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\{'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth"))
D:\Facultad\AA\aprendizaje_automatico\TP2\pipline_comparar.py:75: SyntaxWarning: invalid escape sequence '\m'
  modelo.load_state_dict(torch.load(f"TP2\models\DictMejores\{nombre_modelo}_best_model.pth"))


KeyboardInterrupt: 